In [1]:
import math
import numpy as np
import pandas as pd
from sympy import var, solveset, S
from sympy import *
from sympy import Symbol
from matplotlib import pyplot as plt

In [2]:
#Input Database

N = 7
Pc = np.array([673.3, 709, 617.4, 550.4, 487, 439.6, 260]) #in Psi
Tc = np.array([342.67, 549.67, 665.77, 765.27, 845.28, 913.392, 1116]) #in Rankine
w = np.array([0.011, 0.099, 0.153, 0.199, 0.251, 0.3, 0.65])
MW = np.array([16.04, 30.07, 44.1, 58.12, 72.15, 86.18, 210])
y = np.array([0.74,0.05,0.04,0.04,0.03,0.02,0.08])
#Input PT Condition
R = 10.731577089016
P = 3000
T = 180 

In [3]:
def ZBB(P,T):
  Pc = np.array([673.3, 709, 617.4, 550.4, 487, 439.6, 260]) #in Psi
  Tc = np.array([342.67, 549.67, 665.77, 765.27, 845.28, 913.392, 1116]) #in Rankine
  y = np.array([0.74,0.05,0.04,0.04,0.03,0.02,0.08])
  Ppc = 0
  Tpc = 0
  for i in range (N):
    Ppc += Pc[i]*y[i]
    Tpc += Tc[i]*y[i]
  Pr = P/Ppc
  Tr = (T+460)/Tpc
  A = 1.39*(Tr-0.92)**0.5-0.36*Tr-0.101
  A1 = (1-(1.39*(Tr-0.92)**0.5-0.36*Tr-0.101))/math.exp((0.62-0.23*Tr)*Pr+(0.066/(Tr-0.86)-0.037)*Pr**2+0.32/10**(9*(Tr-1))*Pr**6)
  A2 = (0.132-0.32*math.log(Tr,10))*Pr**(10**(0.3106-0.49*Tr+0.1824*Tr**2))
  return(A+A1+A2)

In [4]:
A =[]
for i in range (N):
  if w[i] <= 0.49:
    m = 0.37464+1.5422*w[i]-0.26992*w[i]**2
  elif w[i] > 0.49:
    m = 0.379642+1.48503*w[i]-0.164423*w[i]**2+0.016666*w[i]**3
  # a = 0.45724*R*R*Tc[i]*Tc[i]/Pc[i]*(1+m*(1-math.sqrt((T+459.67)/Tc[i])))**2/(R**2*(T+459.67)**2)*P
  a = (1+m*(1-math.sqrt((T+459.67)/Tc[i])))**2
  Ai = 0.45724*(P/Pc[i])/(((T+459.67)/Tc[i])**2)*a
  A.append(Ai)
print(A)

[0.4289707995844514, 1.3129653197262257, 2.4647034318508565, 3.9891409557289816, 5.917539910543638, 8.161942321761563, 27.597494865944444]


In [5]:
B = []
for i in range (N):
  b = 0.0778*(P/Pc[i])/((T+459.67)/Tc[i])
  B.append(b)
# print(B)

In [6]:
sigma_A = 0
sigma_B = 0
for i in range (N):
  for j in range (N):
    sigma_A += y[i]*y[j]*((A[i]*A[j])**0.5)
  sigma_B += y[i]*B[i]
print(sigma_A)
print(sigma_B)

1.5253072890793093
0.3470479269371258


In [7]:
c3 = 1
c2 = -(1-sigma_B)
c1 = sigma_A-3*sigma_B**2-2*sigma_B
c0 = -(sigma_A*sigma_B-sigma_B**2-sigma_B**3)
Z = var('Z', real=True)
result = list(solveset(c3*Z**3+c2*Z**2+c1*Z+c0, Z, domain=S.Reals))
z = result
z

[0.714462374586724]

In [8]:
MWa = 0
for i in range (N):
  MWa += MW[i]*y[i]
print("Specific Gravity (SG) = ", MWa/28.97) 

Specific Gravity (SG) =  1.3168795305488439


In [9]:
Bg = 0.0283* z[0]*(T+459.67)/P
print("Gas Formation Volume Factor =", Bg, "cu-ft/scf")

Gas Formation Volume Factor = 0.00431122338813283 cu-ft/scf


In [10]:
rho_g = 2.7*MWa/28.97*P/(z[0]*(T+459.67))
print("Gas Density =", rho_g, "lbm/cu-ft")

Gas Density = 23.3397242198615 lbm/cu-ft


In [11]:
def vis_g(T, MWa, rho):
  A = ((9.379+0.01607*MWa)*(T+459.67)**1.5)/(209.2+19.26*MWa+(T+459.67))
  B = 3.448+986.4/(T+459.67)+0.01009*MWa
  C = 2.447-0.2224*B
  return(A*exp(B*rho**C)/(10**4))
print("Gas Viscosity=", vis_g(T,MWa,rho_g/62.428), "cP")

Gas Viscosity= 0.0490099238115337 cP


In [12]:
print("Z Factor from Beggs-Brill Correlation is", ZBB(P,T))
print("It deviates for", abs(ZBB(P,T)-z[0])/ZBB(P,T)*100,"%")

Z Factor from Beggs-Brill Correlation is 0.7343039159817604
It deviates for 2.70208846271892 %
